# Tarea 5

In [1]:
# Configuración para GPU

import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix

import optuna
import mlflow
import dagshub

np.random.seed(42)
tf.random.set_seed(42)

2025-11-26 23:48:48.534469: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/enriq/tf-gpu/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Verificar y configurar GPU

gpus = tf.config.list_physical_devices('GPU')
print(f"GPUs disponibles: {len(gpus)}")

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU configurada: {gpus[0].name}")
    except RuntimeError as e:
        print(f"Error al configurar GPU: {e}")

GPUs disponibles: 1
GPU configurada: /physical_device:GPU:0


In [4]:
# Dagshub
dagshub.init(repo_owner='404brainnotfound-ai', repo_name='Tarea_5', mlflow=True)
print(f"URI de seguimiento: {mlflow.get_tracking_uri()}")

Accessing as 404brainnotfound-ai

Initialized MLflow to track repo "404brainnotfound-ai/Tarea_5"

Repository 404brainnotfound-ai/Tarea_5 initialized!

URI de seguimiento: https://dagshub.com/404brainnotfound-ai/Tarea_5.mlflow


In [5]:
# Dataset
df = pd.read_csv('global_house_purchase_dataset.csv')
df = df.drop('property_id', axis=1)

print(f"Forma: {df.shape}")
print(f"Clases: {df['decision'].value_counts().to_dict()}")

Forma: (200000, 24)
Clases: {0: 153932, 1: 46068}


In [6]:
# Codificación one hot para variables
columnas_categoricas = ['country', 'city', 'property_type', 'furnishing_status']
df_codificado = pd.get_dummies(df, columns=columnas_categoricas, dtype=int)

print(f"Características después de codificar: {df_codificado.shape[1] - 1}")

Características después de codificar: 81


In [7]:
# Separar X, y
X = df_codificado.drop('decision', axis=1).values
y = df_codificado['decision'].values

# Normalizar
escalador = MinMaxScaler()
X_escalado = escalador.fit_transform(X)

print(f"Forma de X: {X_escalado.shape}")
print(f"Forma de y: {y.shape}")

Forma de X: (200000, 81)
Forma de y: (200000,)


In [9]:
# División 70/20/10
X_entreno, X_temp, y_entreno, y_temp = train_test_split(X_escalado, y, test_size=0.30, random_state=42, stratify=y)
X_prueba, X_validacion, y_prueba, y_validacion = train_test_split(X_temp, y_temp, test_size=1/3, random_state=42, stratify=y_temp)

print(f"Entreno: {len(X_entreno)} ({len(X_entreno)/len(X)*100:.1f}%)")
print(f"Prueba:  {len(X_prueba)} ({len(X_prueba)/len(X)*100:.1f}%)")
print(f"Validación: {len(X_validacion)} ({len(X_validacion)/len(X)*100:.1f}%)")

num_caracteristicas = X_entreno.shape[1]
print(f"\nNúmero de características: {num_caracteristicas}")

Entreno: 140000 (70.0%)
Prueba:  40000 (20.0%)
Validación: 20000 (10.0%)

Número de características: 81


In [10]:
def crear_modelo(trial):
    
    # Hiperparámetros
    n_capas = trial.suggest_int("n_capas", 2, 4)
    unidades_1 = trial.suggest_int("unidades_1", 128, 512, step=128)
    unidades_2 = trial.suggest_int("unidades_2", 64, 256, step=64)
    dropout = trial.suggest_float("dropout", 0.2, 0.5)
    valor_l2 = trial.suggest_float("valor_l2", 1e-6, 1e-3, log=True)
    tasa_aprendizaje = trial.suggest_float("tasa_aprendizaje", 1e-5, 1e-3, log=True)
    
    # Modelo Dense
    modelo = keras.Sequential()
    
    # Primera capa
    modelo.add(layers.Dense(unidades_1, activation='relu', 
                           kernel_regularizer=regularizers.l2(valor_l2),
                           input_shape=(num_caracteristicas,)))
    modelo.add(layers.BatchNormalization())
    modelo.add(layers.Dropout(dropout))
    
    # Capas intermedias
    for i in range(n_capas - 1):
        modelo.add(layers.Dense(unidades_2, activation='relu',
                               kernel_regularizer=regularizers.l2(valor_l2)))
        modelo.add(layers.BatchNormalization())
        modelo.add(layers.Dropout(dropout))
    
    # Capa de salida
    modelo.add(layers.Dense(1, activation='sigmoid'))
    
    optimizador = keras.optimizers.Adam(learning_rate=tasa_aprendizaje)
    modelo.compile(optimizer=optimizador, loss='binary_crossentropy', metrics=['accuracy'])
    
    return modelo

In [11]:
def objetivo(trial):
    modelo = crear_modelo(trial)
    
    tamano_lote = 128
    
    callbacks = [
        EarlyStopping(monitor='loss', patience=5, restore_best_weights=True, verbose=0)]
    
    # Entrenar solo con entreno
    historial = modelo.fit(
        X_entreno, y_entreno,
        batch_size=tamano_lote,
        epochs=15,
        callbacks=callbacks,
        verbose=0)
    
    # Evaluar en prueba
    perdida, precision = modelo.evaluate(X_prueba, y_prueba, verbose=0)
    
    # Liberar memoria
    del modelo
    keras.backend.clear_session()
    
    return perdida

In [12]:
# Optimización con Optuna
mlflow.set_experiment("Tarea5_Optuna")

estudio = optuna.create_study(direction="minimize", study_name="prediccion_casa")
estudio.optimize(objetivo, n_trials=30, show_progress_bar=True)

print(f"\nMejor intento: {estudio.best_trial.number}")
print(f"Mejor pérdida: {estudio.best_value:.6f}")
print(f"\nMejores parámetros:")
for clave, valor in estudio.best_params.items():
    print(f"  {clave}: {valor}")

[I 2025-11-26 23:58:05,777] A new study created in memory with name: prediccion_casa
  0%|          | 0/30 [00:00<?, ?it/s]/home/enriq/tf-gpu/lib/python3.11/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1764223086.240848    3260 gpu_process_state.cc:208] Using CUDA malloc Async allocator for GPU: 0
I0000 00:00:1764223086.491939    3260 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2857 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1
2025-11-26 23:58:15.009252: I external/local_xla/xla/service/service.cc:163] XLA service 0x706a7c007610 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:

[I 2025-11-27 00:01:25,394] Trial 0 finished with value: 0.022582845762372017 and parameters: {'n_capas': 4, 'unidades_1': 128, 'unidades_2': 256, 'dropout': 0.46567924867624183, 'valor_l2': 8.405452643476421e-05, 'tasa_aprendizaje': 0.0007820929266378541}. Best is trial 0 with value: 0.022582845762372017.


Best trial: 1. Best value: 0.00735917:   7%|▋         | 2/30 [05:59<1:22:16, 176.29s/it]

[I 2025-11-27 00:04:05,364] Trial 1 finished with value: 0.007359170354902744 and parameters: {'n_capas': 2, 'unidades_1': 256, 'unidades_2': 64, 'dropout': 0.37949437413517373, 'valor_l2': 5.991447875098912e-06, 'tasa_aprendizaje': 0.0004039905621101411}. Best is trial 1 with value: 0.007359170354902744.


Best trial: 1. Best value: 0.00735917:  10%|█         | 3/30 [08:31<1:14:19, 165.15s/it]

...

[I 2025-11-27 01:33:09,823] Trial 27 finished with value: 0.010596007108688354 and parameters: {'n_capas': 3, 'unidades_1': 512, 'unidades_2': 128, 'dropout': 0.47825675151390334, 'valor_l2': 1.6325719587945135e-06, 'tasa_aprendizaje': 0.00011974965871693592}. Best is trial 13 with value: 0.0065401094034314156.


Best trial: 13. Best value: 0.00654011:  97%|█████████▋| 29/30 [1:37:24<02:24, 144.00s/it]

[I 2025-11-27 01:35:30,131] Trial 28 finished with value: 0.009974529035389423 and parameters: {'n_capas': 3, 'unidades_1': 384, 'unidades_2': 192, 'dropout': 0.39985547140091854, 'valor_l2': 3.409723705361805e-06, 'tasa_aprendizaje': 0.00014723438989321553}. Best is trial 13 with value: 0.0065401094034314156.


Best trial: 13. Best value: 0.00654011: 100%|██████████| 30/30 [1:39:33<00:00, 199.12s/it]

[I 2025-11-27 01:37:39,304] Trial 29 finished with value: 0.07912696897983551 and parameters: {'n_capas': 4, 'unidades_1': 384, 'unidades_2': 64, 'dropout': 0.44431475078038163, 'valor_l2': 1.3977784489449164e-06, 'tasa_aprendizaje': 8.375952042574526e-05}. Best is trial 13 with value: 0.0065401094034314156.

Mejor intento: 13
Mejor pérdida: 0.006540

Mejores parámetros:
  n_capas: 3
  unidades_1: 384
  unidades_2: 64
  dropout: 0.48193138506574673
  valor_l2: 1.1721494032702744e-06
  tasa_aprendizaje: 0.00037509832294593576
